In [1]:
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent))

from datetime import date, timedelta
import codecs
import pandas as pd
from os.path import join

from bulletin import default_input, default_output
from bulletin.services.metabase import Metabase
from bulletin.casos_confirmados import CasosConfirmados
from bulletin.utils.static import Municipios
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash, normalize_number
from bulletin.utils import static
from bulletin.utils.xls_ import fit_cols

from datetime import datetime

pd.set_option('display.max_columns', 500)


---------------------------------------------------------------

root: /home/vinnylg/projects/covid19datascience/bulletin
parent: /home/vinnylg/projects/covid19datascience
default_input: /home/vinnylg/projects/covid19datascience/input
default_output: /home/vinnylg/projects/covid19datascience/output

---------------------------------------------------------------



In [2]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'].apply(normalize_text) + '/' + municipios['uf']

In [3]:
cc = CasosConfirmados()
aaaaaa = cc.read_excel()

databases: ['cc_21_05_2021', 'cc_22_05_2021', 'cc_23_05_2021', 'cc_25_05_2021']
reading Casos Confirmados xlsx time elepsed:00:03:36


In [4]:
aaa = aaaaaa.sort_values(['comunicacao','nome','idade']).copy()
aaa = aaa[[  'id_notifica', 'ibge_resid', 'ibge_atend',
             'nome', 'sexo', 'idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is',
             'evolucao', 'data_evolucao', 'data_com_evolucao']]
aaa

,id_notifica,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao
0,-1,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,LACEN,2020-03-12,2020-03-12 00:00:00,NaN,CURA,NaT,NaT
1,-1,410690,410690,FLAMARION PETRIV DE ABREU,M,43,LACEN,2020-03-12,2020-03-12 00:00:00,2020-03-07 00:00:00,CURA,NaT,NaT
2,-1,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,LACEN,2020-03-12,2020-03-12 00:00:00,2020-03-08 00:00:00,CURA,NaT,NaT
3,-1,410690,410690,LUIZ ANTONIO CAXAMBU FREITAS,M,54,GENOPRIMER,2020-03-11,2020-03-12 00:00:00,2020-03-09 00:00:00,CURA,NaT,NaT
4,-1,410690,410690,MARIANA CAXAMBU FREITAS,F,25,GENOPRIMER,2020-03-11,2020-03-12 00:00:00,2020-03-01 00:00:00,CURA,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079250,3898732,411820,411820,ZILFREDO WAGNER,M,57,"COVID-19, Biologia Molecular",2021-05-26,27/05/2021,14/05/2021,ATIVO,NaT,NaT
1079251,3828992,411570,411570,ZITA SOUZA DE CAMPOS,F,71,"COVID-19, Biologia Molecular",2021-05-26,27/05/2021,NaN,ATIVO,NaT,NaT
1079252,3784533,411790,999999,ZIVANILDO SILVA CORDEIRO,M,36,"COVID-19, Biologia Molecular",2021-05-25,27/05/2021,20/05/2021,ATIVO,NaT,NaT
1079253,3831356,410690,410690,ZORAIDE DOS REIS RIBEIRO,F,72,"COVID-19, Biologia Molecular",2021-05-26,27/05/2021,22/05/2021,ATIVO,NaT,NaT


In [5]:
aaa = pd.merge(aaa.rename(columns={'ibge_resid':'ibge'}),municipios,how='left',on='ibge').rename(columns={'uf':'uf_resid', 'ibge':'ibge_resid'})
aaa

,id_notifica,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,ibge7,rs,regional,macro,municipio,uf_resid,populacao,estado,mun_resid
0,-1,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,LACEN,2020-03-12,2020-03-12 00:00:00,NaN,CURA,NaT,NaT,4105508,13,Cianorte,Noroeste,Cianorte,PR,83816,Paraná,CIANORTE
1,-1,410690,410690,FLAMARION PETRIV DE ABREU,M,43,LACEN,2020-03-12,2020-03-12 00:00:00,2020-03-07 00:00:00,CURA,NaT,NaT,4106902,2,Metropolitana,Leste,Curitiba,PR,1948626,Paraná,CURITIBA
2,-1,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,LACEN,2020-03-12,2020-03-12 00:00:00,2020-03-08 00:00:00,CURA,NaT,NaT,4106902,2,Metropolitana,Leste,Curitiba,PR,1948626,Paraná,CURITIBA
3,-1,410690,410690,LUIZ ANTONIO CAXAMBU FREITAS,M,54,GENOPRIMER,2020-03-11,2020-03-12 00:00:00,2020-03-09 00:00:00,CURA,NaT,NaT,4106902,2,Metropolitana,Leste,Curitiba,PR,1948626,Paraná,CURITIBA
4,-1,410690,410690,MARIANA CAXAMBU FREITAS,F,25,GENOPRIMER,2020-03-11,2020-03-12 00:00:00,2020-03-01 00:00:00,CURA,NaT,NaT,4106902,2,Metropolitana,Leste,Curitiba,PR,1948626,Paraná,CURITIBA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079250,3898732,411820,411820,ZILFREDO WAGNER,M,57,"COVID-19, Biologia Molecular",2021-05-26,27/05/2021,14/05/2021,ATIVO,NaT,NaT,4118204,1,Paranaguá,Leste,Paranaguá,PR,156174,Paraná,PARANAGUA
1079251,3828992,411570,411570,ZITA SOUZA DE CAMPOS,F,71,"COVID-19, Biologia Molecular",2021-05-26,27/05/2021,NaN,ATIVO,NaT,NaT,4115705,1,Paranaguá,Leste,Matinhos,PR,35219,Paraná,MATINHOS
1079252,3784533,411790,999999,ZIVANILDO SILVA CORDEIRO,M,36,"COVID-19, Biologia Molecular",2021-05-25,27/05/2021,20/05/2021,ATIVO,NaT,NaT,4117909,20,Toledo,Oeste,Palotina,PR,32121,Paraná,PALOTINA
1079253,3831356,410690,410690,ZORAIDE DOS REIS RIBEIRO,F,72,"COVID-19, Biologia Molecular",2021-05-26,27/05/2021,22/05/2021,ATIVO,NaT,NaT,4106902,2,Metropolitana,Leste,Curitiba,PR,1948626,Paraná,CURITIBA


In [6]:
aaa.loc[aaa['evolucao']=='OBITO', 'evolucao']=2
aaa.loc[aaa['evolucao']=='CURA', 'evolucao']=1
aaa.loc[aaa['evolucao']=='ATIVO', 'evolucao']=3
aaa.groupby('evolucao')[['sexo']].count()

,sexo
evolucao,
1,733329
2,25966
3,319960


In [9]:
aaa['identificacao'] = 0
cc.df = aaa[cc.default_columns]
cc.save('cc_27_05_2021')

Load /home/vinnylg/projects/covid19datascience/bulletin/database/casos_confirmados/cc_27_05_2021.pkl
saving Casos Confirmados to pkl time elepsed:00:00:01
